# EDA

In [ ]:
import os
import pandas as pd
import snowflake.connector
from dotenv import load_dotenv  
from scipy import stats
import numpy as np

In [ ]:
load_dotenv('../.env')

# Connect to Snowflake
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    warehouse="COMPUTE_WH",
    database="DYNAMIC_PRICING",
    schema="MARTS"
)

print('Connection established')

In [ ]:
# 1. Define the Query
# I want the Full Log (Granular Data) for Machine Learning
query = """
SELECT * FROM DYNAMIC_PRICING.MARTS.MART_FULL_SALES_LOG 
ORDER BY SALES_DATE DESC
"""

# 2. Run Query & Load into Pandas
# cursor().execute() runs the query, fetchall() gets the data
cur = conn.cursor()
cur.execute(query)
data = cur.fetchall()

# 3. Get Column Names (so the DataFrame isn't just numbers)
col_names = [desc[0] for desc in cur.description]

# 4. Create the DataFrame
df = pd.DataFrame(data, columns=col_names)

# 5. Close Connection (Good practice!)
cur.close()
conn.close()

# 6. Verify
print(f"Success! Loaded {df.shape[0]} rows and {df.shape[1]} columns.")
df.head()

In [ ]:
print("--- Data Info ---")
df.info()

In [ ]:
# Summary statistics for Prices and Weather
# I filter for relevant columns to avoid noise
stats_cols = ['OUR_PRICE', 'COMPETITOR_PRICE', 'TEMPERATURE_C', 'PRECIPITATION_MM', 'PRICE_DIFF']

# Describe generates the stats
print(df[stats_cols].describe().round(2))

In [ ]:
# Count unique orders per category
print("--- Sales by Category ---")
print(df['CATEGORY_NAME'].value_counts())

# Count weather conditions
print("\n--- Weather Conditions ---")
print(df['WEATHER_CONDITION'].value_counts())

In [ ]:
# Check the start and end dates
print(f"Data Start Date: {df['SALES_DATE'].min()}")
print(f"Data End Date:   {df['SALES_DATE'].max()}")
print(f"Total Days:      {(df['SALES_DATE'].max() - df['SALES_DATE'].min()).days}")

In [ ]:
# 1. Define the "Normal" Range (IQR Rule)
Q1 = df['OUR_PRICE'].quantile(0.25)
Q3 = df['OUR_PRICE'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 2. Identify the Outliers
outliers = df[(df['OUR_PRICE'] < lower_bound) | (df['OUR_PRICE'] > upper_bound)]

print(f"--- Outlier Report ---")
print(f"Normal Price Range: ${lower_bound:.2f} to ${upper_bound:.2f}")
print(f"Number of Outliers detected: {len(outliers)}")
print(f"Percentage of data that is 'Anomalous': {(len(outliers)/len(df))*100:.2f}%")

# 3. Who are they? (Top 5 Extreme Cases)
print("\n--- Top 5 Most Expensive 'Anomalies' ---")
print(outliers.nlargest(5, 'OUR_PRICE')[['SALES_DATE', 'CATEGORY_NAME', 'OUR_PRICE', 'WEATHER_CONDITION']])

In [ ]:
# 1. Split data into two groups: Rainy Days vs Clear Days
rainy_sales = df[df['WEATHER_CONDITION'] == 'Rain']['OUR_PRICE']
clear_sales = df[df['WEATHER_CONDITION'] == 'Clear']['OUR_PRICE']

# 2. Run the T-Test (Independent samples)
t_stat, p_val = stats.ttest_ind(rainy_sales, clear_sales, equal_var=False)

print(f"--- Hypothesis Test: Rain vs Clear ---")
print(f"Rainy Days Mean Revenue: ${rainy_sales.mean():.2f}")
print(f"Clear Days Mean Revenue: ${clear_sales.mean():.2f}")
print(f"P-Value: {p_val:.5f}")

if p_val < 0.05:
    print("RESULT: Statistically Significant! Rain DOES affect sales.")
else:
    print("RESULT: Not Significant. The difference could be random chance.")

In [ ]:
# I look at 'PRICE_DIFF' (My Price - Competitor)
price_diffs = df['PRICE_DIFF'].dropna() # Remove nulls just in case

# Calculate 95% Confidence Interval
conf_level = 0.95
degrees_freedom = len(price_diffs) - 1
sample_mean = np.mean(price_diffs)
sample_sem = stats.sem(price_diffs) # Standard Error of Mean

confidence_interval = stats.t.interval(conf_level, degrees_freedom, sample_mean, sample_sem)

print(f"\n--- 95% Confidence Interval for Price Difference ---")
print(f"Sample Mean: ${sample_mean:.2f}")
print(f"Range: [${confidence_interval[0]:.2f}, ${confidence_interval[1]:.2f}]")
print("Interpretation: I am 95% confident the 'True' price difference falls in this range.")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select numeric columns only
corr_cols = ['OUR_PRICE', 'COMPETITOR_PRICE', 'TEMPERATURE_C', 'PRECIPITATION_MM', 'PRICE_DIFF']
corr_matrix = df[corr_cols].corr()

# Plot
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix: What drives what?")
plt.show()

In [ ]:
import plotly.figure_factory as ff
import numpy as np

# 1. Prepare data (Drop nulls just in case)
v1 = df['OUR_PRICE'].dropna()
v2 = df['COMPETITOR_PRICE'].dropna()

# 2. Create Distribution Plot (Histogram + KDE Curve)
hist_data = [v1, v2]
group_labels = ['My Price', 'Competitor Price']

# Create the plot with custom colors
fig = ff.create_distplot(hist_data, group_labels, 
                         bin_size=25, 
                         show_hist=True, 
                         show_rug=False,
                         colors=['#00D4FF', '#FF5733']) # Blue vs Red

fig.update_layout(title_text='Price Distribution: Me vs. Them')
fig.show()

In [ ]:
import plotly.express as px

fig = px.box(df, 
             x="WEATHER_CONDITION", 
             y="OUR_PRICE", 
             color="WEATHER_CONDITION",
             points="all", # Show actual data points too
             title="Revenue Distribution by Weather",
             labels={"OUR_PRICE": "Revenue per Order ($)"})

fig.show()

In [ ]:
fig = px.strip(df, 
               x="CATEGORY_NAME", 
               y="OUR_PRICE", 
               color="CATEGORY_NAME", 
               title="Price Consistency by Category")

fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# Extract Day of Week (0=Monday, 6=Sunday)
df['day_of_week'] = pd.to_datetime(df['SALES_DATE']).dt.day_name()

# Sort order for the chart
order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

fig = px.box(df, 
             x="day_of_week", 
             y="OUR_PRICE", 
             category_orders={"day_of_week": order},
             title="Sales Distribution by Day of Week",
             color="day_of_week")

fig.show()

In [ ]:
import plotly.express as px

# 1. Group by Date to get Total Daily Revenue
daily_revenue = df.groupby('SALES_DATE')['OUR_PRICE'].sum().reset_index()

# 2. Plot the Trend
fig = px.line(daily_revenue, 
              x='SALES_DATE', 
              y='OUR_PRICE',
              title='Daily Revenue Trend (2016-2018 Data Shifted)',
              labels={'OUR_PRICE': 'Total Revenue ($)', 'SALES_DATE': 'Date'})

# Add a "Trendline" to see the general direction
fig.update_traces(line_color='#00D4FF', line_width=2)
fig.show()

In [ ]:
# 1. Sum Revenue by Category
category_perf = df.groupby('CATEGORY_NAME')['OUR_PRICE'].sum().reset_index()
category_perf = category_perf.sort_values('OUR_PRICE', ascending=True) # Sort for better chart

# 2. Horizontal Bar Chart
fig = px.bar(category_perf, 
             x='OUR_PRICE', 
             y='CATEGORY_NAME',
             orientation='h', # Horizontal is easier to read
             title='Total Revenue by Category',
             text_auto='.2s', # Show values like "10k"
             color='OUR_PRICE',
             color_continuous_scale='Bluered')

fig.show()

In [ ]:
# 1. Pivot Table: Rows=Category, Cols=Weather, Values=Average Daily Revenue
# I aggregate by MEAN to account for different numbers of rainy vs clear days
weather_pivot = df.pivot_table(index='CATEGORY_NAME', 
                               columns='WEATHER_CONDITION', 
                               values='OUR_PRICE', 
                               aggfunc='mean').reset_index()

# 2. Calculate the "Rain Lift" (How much better/worse is Rain?)
# Rain Lift = (Rain Revenue - Clear Revenue) / Clear Revenue
weather_pivot['Rain_Lift'] = (weather_pivot['Rain'] - weather_pivot['Clear']) / weather_pivot['Clear']

# 3. Plot the Lift
fig = px.bar(weather_pivot, 
             x='Rain_Lift', 
             y='CATEGORY_NAME',
             orientation='h',
             title='Impact of Rain on Revenue (Percentage Lift)',
             labels={'Rain_Lift': 'Revenue Change in Rain (%)'},
             color='Rain_Lift',
             color_continuous_scale='RdBu', # Red=Bad, Blue=Good
             text_auto='.1%')

# Add a line at 0% (No impact)
fig.add_vline(x=0, line_dash="dash", line_color="black")
fig.show()

In [ ]:
# 1. Select the columns that actually matter
cols_to_plot = ['OUR_PRICE', 'COMPETITOR_PRICE', 'TEMPERATURE_C', 'PRECIPITATION_MM']

# 2. Create the Pairplot
# Hue='WEATHER_CONDITION' lets me see if Rain creates distinct "clusters" of data
plt.figure(figsize=(10, 8))
sns.pairplot(df, vars=cols_to_plot, hue='WEATHER_CONDITION', palette='husl', plot_kws={'alpha': 0.5})
plt.title("Multivariate Interactions")
plt.show()

In [ ]:
from pandas.plotting import lag_plot

# Resample to Daily Total Revenue (since individual orders are noisy)
daily_sales = df.groupby('SALES_DATE')['OUR_PRICE'].sum()

plt.figure(figsize=(10, 5))

# Plot Lag 1 (Yesterday vs Today)
plt.subplot(1, 2, 1)
lag_plot(daily_sales, lag=1)
plt.title("Lag 1 (Yesterday vs Today)")

# Plot Lag 7 (Last Week vs Today)
plt.subplot(1, 2, 2)
lag_plot(daily_sales, lag=7)
plt.title("Lag 7 (Same Day Last Week)")

plt.tight_layout()
plt.show()